# Questions

 #### - How do we know where to locate the photosphere in the height that is set by the nz value?
 #### - Which units should we use for the figures?
 #### Equivalent of megameters for the nx and ny pixels -> ny is the vertical component

 #### - Why is that the magnetic field z figure looks the same as the eps figure?

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.nddata.utils import Cutout2D
import matplotlib.cm as cm
import matplotlib as mpl
import astropy.units as u
from collections import Counter
import numpy as np 
from keras.models import Sequential #sequential model of layers, as the one explained in this part of the book
from keras.layers.core import Dense

In [ ]:

class Read:
    
    def __init__(self):
        
        self.ptm = ''
        self.filename = '154000'
        self.nx = 480
        self.ny = 256
        self.nz = 480
        
    
    def read(self,verbose=1):
        
        ################################
        # Charging the data into the code - every data is converted into a cube 
        # of data so that it has the form of the dominium of the simulation
        #
        # The lines of the code for mvxx, mvyy, mbxx and mbyy are commented beacuse
        # we are not interested in ploting this magnitudes for the image analysis 
        # we are going to make
        #
        # The temperature is obtained from the data file related to the 
        # equation of state (EOS)
        ################################
        
        print("reading IOUT")
        self.iout = np.fromfile(self.ptm+"iout."+self.filename,dtype=np.float32)
        self.iout = np.reshape(self.iout, (self.nx, self.nz), order="A")
        print(np.shape(self.iout))
        print("IOUT done")   
        
        print("reading EOS")
        self.mtpr = np.fromfile(self.ptm+"eos."+self.filename,dtype=np.float32)
        self.mtpr = np.reshape(self.mtpr, (2, self.nx,self.ny,self.nz), order="A")
        n_eos = 0
        self.mtpr = self.mtpr[n_eos,:,:,:] 
        # n_eos -> 0: temperature ; 1: pressure
        print("EOS done")

        #print(self.mtpr[0,0,0,0],self.mtpr[1,0,0,0])

        print("reading rho")
        self.mrho = np.fromfile(self.ptm+"result_0."+self.filename,dtype=np.float32)
        self.mrho = np.reshape(self.mrho, (self.nx,self.ny,self.nz), order="A")
        print(np.shape(self.mrho))
        print("rho done")

#         print("reading vxx")
#         self.mvxx = np.fromfile(self.ptm+"result_1."+self.filename,dtype=np.float32)
#         self.mvxx = np.reshape(self.mvxx,(self.nx,self.nz,self.ny),order="C")
#         print("vxx done")

        print("reading vyy")
        self.mvyy = np.fromfile(self.ptm+"result_2."+self.filename,dtype=np.float32)
        self.mvyy = np.reshape(self.mvyy,(self.nx,self.ny,self.nz),order="C")
        print("vyy done")

#         print("reading vzz")
#         self.mvzz = np.fromfile(self.ptm+"result_3."+self.filename,dtype=np.float32)
#         self.mvzz = np.reshape(self.mvzz,(self.nx,self.nz, self.ny),order="A")
#         print(np.shape(self.mvzz))
#         print("vzz done")
        
        print("reading eps")
        self.eps = np.fromfile(self.ptm+"result_4."+self.filename,dtype=np.float32)
        self.eps = np.reshape(self.eps,(self.nx,self.ny,self.nz),order="C")
        print("eps done")

#         print("reading bxx")
#         self.mbxx = np.fromfile(self.ptm+"result_5."+self.filename,dtype=np.float32)
#         self.mbxx = np.reshape(self.mbxx,(self.nx,self.nz,self.ny),order="C")
#         print("bxx done")

        print ("reading byy")
        self.mbyy = np.fromfile(self.ptm+"result_6."+self.filename,dtype=np.float32)
        self.mbyy = np.reshape(self.mbyy,(self.nx,self.ny,self.nz),order="C")
        print("byy done")

#         print("reading bzz")
#         self.mbzz = np.fromfile(self.ptm+"result_7."+self.filename,dtype=np.float32)
#         self.mbzz = np.reshape(self.mbzz,(self.nx,self.nz, self.ny),order="A")
#         print(np.shape(self.mbzz))
#         print("bzz done")
        
        #############################################################
        #Converting the data into cgs units (if I'm not wrong)
        #############################################################

#         self.mvxx=self.mvxx/self.mrho
        self.mvyy=self.mvyy/self.mrho
#         self.mvzz=(self.mvzz/self.mrho)*-6e10
        coef = np.sqrt(4.0*np.pi)
#         self.mbxx=self.mbxx*coef
        self.mbyy=self.mbyy*coef
#         self.mbzz=self.mbzz*coef

        
        #############################################################
        # Here we select the slices for a nz=cte so that we obtain the 2d graphs of 
        # the intensity, temperature, magnetic field in z and velocity in z
        #############################################################v
        
        plot_mbyy, plot_iout, plot_mvyy, plot_mrho, plot_mtpr, plot_eps = np.zeros((self.nx, self.nz)), np.zeros((self.nx, self.nz)), np.zeros((self.nx, self.nz)), np.zeros((self.nx, self.nz)), np.zeros((self.nx, self.nz)), np.zeros((self.nx, self.nz))
        
        ny_slice = 170
        
#         for i in range(np.shape(self.mbzz)[0]):
        plot_mbyy = self.mbyy[:,ny_slice,:]
        plot_mvyy = self.mvyy[:,ny_slice,:]
        plot_mtpr = self.mtpr[:,ny_slice,:]
        plot_eps = self.eps[:,ny_slice,:]
        plot_iout = self.iout
        
        
#         plot_iout = np.reshape(plot_iout, (self.nx, self.ny))
#         plot_mbzz = np.reshape(plot_mbzz, (np.shape(self.mbzz)[2], np.shape(self.mbzz)[2]))
#         plot_mvzz = np.reshape(plot_mvzz, (np.shape(self.mvzz)[2], np.shape(self.mvzz)[2]))
#         plot_mtpr = np.reshape(plot_mtpr, (np.shape(self.mtpr)[0], np.shape(self.mtpr)[2]))
#         plot_eps = np.reshape(plot_eps, (np.shape(self.eps)[0], np.shape(self.eps)[2]))
        
        #############################################################
        # Here we plot the 2D graphs using Matplotlib
        #############################################################
        
        f = plt.figure(figsize=(15,15))

        ax1 = f.add_subplot(231)
        by = ax1.imshow(plot_mbyy, cmap='gist_gray', origin='lower')
        ax1.set_title('magnetic field in y')
        divider1 = make_axes_locatable(ax1)
        cax1 = divider1.append_axes('top', size='5%', pad=0.8)
        f.colorbar(by, cax=cax1, orientation="horizontal", label="NN")
        ax1.grid()

        ax2 = f.add_subplot(232)
        surf = ax2.imshow(plot_iout, cmap='gist_gray', origin='lower')
        ax2.set_title('Intensity')
        divider2 = make_axes_locatable(ax2)
        cax2 = divider2.append_axes('top', size='5%', pad=0.8)
        f.colorbar(surf, cax=cax2, orientation="horizontal", label="NN")
        ax2.grid()

        ax3 = f.add_subplot(233)
        divider3 = make_axes_locatable(ax3)
        vy = ax3.imshow(plot_mvyy, cmap='gist_gray', interpolation='gaussian', origin='lower')
        cax3 = divider3.append_axes('top', size='5%', pad=0.8)
        ax3.set_title('Velocity in y')
        f.colorbar(vy, cax=cax3, orientation="horizontal", label="NN")
        ax3.grid()

        ax4 = f.add_subplot(234)
        
        divider4 = make_axes_locatable(ax4)
        cax4 = divider4.append_axes('top', size='5%', pad=0.8, label='a')
        data = plot_mtpr
        extent = (np.min(data), np.max(data), np.min(data), np.max(data))
        position = (0,0)      
#         size = u.Quantity((720, 720), u.pixel)
#         cutout = Cutout2D(data, position, size, mode='trim')
        tem = ax4.imshow(data, origin='lower',   cmap='gist_gray', interpolation='gaussian', vmax=np.max(data), extent=extent)
        ax4.set_title('Temperature')
        f.colorbar(tem, cax=cax4, orientation="horizontal", label="NN")
        ax4.grid()
        
        ax5 = f.add_subplot(235)
        eps = ax5.imshow(plot_eps, cmap='gist_gray', origin='lower')
        ax5.set_title('eps')
        divider5 = make_axes_locatable(ax5)
        cax5 = divider5.append_axes('top', size='5%', pad=0.8)
        f.colorbar(eps, cax=cax5, orientation="horizontal", label="NN")
        ax5.grid()

        
        plt.savefig("test1")
        plt.show()
        
        #############################################################
        # This functions are just for some testing, but are not part 
        # of the main code
        #############################################################
        
        def make_box(Npx, **kwargs):
            """Draws a box containing Npx pixels in center of an image"""
            box_size = Npx
            box_halflen = np.ceil(np.sqrt(box_size)/2.)
            mid_pix = plot_iout.shape[0] / 2
            box_halflen, mid_pix
            box = mpl.patches.Rectangle( (mid_pix - box_halflen, mid_pix - box_halflen),
                                       2*box_halflen, 2*box_halflen, fill=False, **kwargs)
            return box
        
        imparams = dict(origin='lower', interpolation='bicubic')
        sequential = dict(**imparams, cmap='inferno')
        diverging = dict(**imparams, cmap='seismic_r')
        magnetogram = dict(**imparams, cmap='gray')
        
        def add_boxes(ax, **kwargs):
            """Adds boxes to an axis"""
            # dashed lines: boxes containing 1e5, 1e6 pixels
            for i in (1, 10):
                ax.add_patch(make_box(i*1e5, **kwargs, ls='--'))
            # solid lines: boxes containing 2**n * 1e5 pixels
            for i in (2, 4, 8, 16):   
                ax.add_patch(make_box(i*1e5, **kwargs))
                
        TARGET_VARIABLE = "user_action"
        TRAIN_TEST_SPLIT = 0.5
        HIDDEN_LAYER_SIZE = 30
        raw_data = [self.mbyy[:,:,:],
                    self.mvyy[:,:,:],
                    self.mtpr[:,:,:],
                    self.iout]

        mask = np.random.rand(self.ny) < TRAIN_TEST_SPLIT

        tr_dataset = [raw_data[0][:,:,mask], raw_data[1][:,:,mask], raw_data[2][:,:,mask]]
        te_dataset = raw_data[~mask] # ~mask is the negation of the truth values of mask

        tr_data = np.array(raw_data.drop(TARGET_VARIABLE, axis=1)) #The values or the training set
        tr_labels = np.array(raw_data[[TARGET_VARIABLE]]) #the values from the column of targets
        te_data = np.array(te_dataset.drop(TARGET_VARIABLE, axis = 1)) #The values for the test set
        te_labels = np.array(te_dataset[[TARGET_VARIABLE]]) #the values from the column of targets

        ffnn = Sequential()
        ffnn.add(Dense(HIDDEN_LAYER_SIZE, input_shape = (3,), activation = "sigmoid"))
        ffnn.add(Dense(1, activation="sigmoid"))
        ffnn.compile(loss="mean_squared_error", optimizer="sgd", metrics=['accuracy'])
        ffnn.fit(tr_data, tr_labels, epochs=150, batch_size=2, verbose=1)

        metrics = ffnn.evaluate(te_data, te_labels, verbose=1)
        print("%s: %2.f%%" % (ffnn.metrics_names[1], metrics[1]*100))
        
if __name__ == "__main__":
    execute = Read()
    execute.read()

reading IOUT
(480, 480)
IOUT done
reading EOS
EOS done
reading rho
(480, 256, 480)
rho done
reading vyy
vyy done
reading eps
eps done
reading byy
